# Deep Learning Tutorial - Modeling

In [1]:
import numpy as np
import pprint

In [18]:
INPUT_SIZE = 2
LAYER_SIZES = [5, 5, 1]
LAYER_SIZES.insert(0, INPUT_SIZE)
LAYER_ACTIVATIONS = ['relu', 'relu', 'sigmoid']

In [36]:
def initialize_network():
    architecture = {}
    # for i, layer_index in enumerate(range(1, len(LAYER_SIZES))):
    for layer in range(1, len(LAYER_SIZES)):
        architecture[f'layer_{layer}'] = {
            'w': np.random.randn(LAYER_SIZES[layer],
                                 LAYER_SIZES[layer-1]) * 1,
            'b': np.zeros(layer),
            'activation': LAYER_ACTIVATIONS[layer-1]
        }
    return architecture

In [37]:
network = initialize_network()
pprint.pprint(network)

{'layer_1': {'activation': 'relu',
             'b': array([0.]),
             'w': array([[ 0.62983372,  0.56878216],
       [-1.47626373,  1.39094761],
       [ 0.19902173,  0.89195303],
       [ 0.96619329,  0.09791707],
       [ 0.77621919, -0.36811113]])},
 'layer_2': {'activation': 'relu',
             'b': array([0., 0.]),
             'w': array([[ 1.16681614,  0.66005872,  0.08133706, -0.54319681,  0.12865929],
       [ 0.17041723,  0.36279056,  1.15376476,  0.14055135,  0.67895405],
       [ 1.39226503, -0.52592959, -0.33414677, -0.4554183 ,  0.93324929],
       [ 0.53377716,  1.03143555, -0.08931775, -1.81003333, -0.95848223],
       [-0.97779417,  0.09039217, -0.02638143, -0.2612654 , -0.325057  ]])},
 'layer_3': {'activation': 'sigmoid',
             'b': array([0., 0., 0.]),
             'w': array([[ 0.64103561,  0.66231714,  0.68292803, -1.99764949,  0.26431266]])}}


In [5]:
def sigmoid_activation(Z):
    activation = 1/(1 + np.exp(-1*Z))
    return activation

In [6]:
def relu_activation(Z):
    activation = np.max(0, Z)
    return activation

In [7]:
def dZ_sigmoid(dA, Z):
    sigmoid = sigmoid_activation(Z)
    dZ = dA * sigmoid * (1 - sigmoid)
    return dZ

In [8]:
def dZ_relu(dA, Z):
    dZ = np.copy(dA)
    dZ[Z <= 0] = 0
    return dZ

In [9]:
act_map = {
    'sigmoid': sigmoid_activation,
    'relu': relu_activation
}

In [10]:
dZ_map = {
    'sigmoid': dZ_sigmoid,
    'relu': dZ_relu
}

In [11]:
def single_forward_pass(A_previous, W, b, activation):
    try:
        act_function = act_map[activation]
    except KeyError:
        print(f'The activation {activation} is not recognized.\nIt must be one of the following: {list(act_map.keys())}')
        return None
    
    Z = np.dot(W, A_previous) + b
    A = act_function(Z)
    
    return A, Z

In [11]:
def full_forward_pass():
    return None

In [12]:
def cross_entropy_cost_function():
    return None

In [13]:
def single_backward_pass():
    return None

In [14]:
def full_backward_pass():
    return None

In [15]:
def train_nn():
    return None

### Resources
This notebook has been inspired by the Towards Data Science post [Let’s code a Neural Network in plain NumPy](https://towardsdatascience.com/lets-code-a-neural-network-in-plain-numpy-ae7e74410795).